In [24]:
import csv
from datetime import datetime, timedelta
import re
import random
import time
import os
import logging
import sys
import errno
import traceback
from pathlib2 import Path
import copy
import ConfigParser
import os

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

print 'all done'

all done


In [31]:
firefox_win_ua = 'Mozilla/5.0 (Windows NT 6.1; rv:52.0) Gecko/20100101 Firefox/52.0'
firefox_mac_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:58.0) Gecko/20100101 Firefox/58.0'
safari_mac_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/604.4.7 (KHTML, like Gecko) Version/11.0.2 Safari/604.4.7'

domain = 'www.prisontalk.com'

headers =   {
                'Host': domain,
                'User-Agent': firefox_win_ua,
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate',
                'DNT': '1',
                'Connection': 'keep-alive'
            }

## THREAD SCRAPER

**NOTE:** *A FEW FORUMS REQUIRE BEING LOGGED IN AND THEREFORE SO WILL THE THREADS OF FORUMS, THIS SCRAPER ONLY WORKS FOR THREADS THAT DO NOT REQUIRE LOGGING IN.  THESE THREADS WILL BE ADDRESS IN ANOTHER NOTEBOOK*

#### SETUP VARIABLES

In [193]:
# GET REMAINING THREADS BY REMOVING ALREADY RETRIEVED THREADS FROM FULL LIST
# remaining_threads = filter_completed_threads(thread_filename, completed_filename, out_path=None)

debug = True

if debug:
    root = '../examples'
    min_time = 4
    max_time = 7
else:
    root = '../data'
    min_time = 8
    max_time = 18

in_path = root + '/threads/thread_lists/'
out_path = root + '/threads/'
log_path = root + '/threads/logs/'

# GET FILE PATHS TO ALL CSV LISTS OF THREADS
any_digit = re.compile(r'\d+')
forums = [str(path) for path in Path(in_path).iterdir() if path.is_file()]
forum_ids = [any_digit.findall(forum_dir)[0] for forum_dir in forums]

take_breaks = False
go_to_bed = False
first_scrape = True
s = create_session(10)

#### SCRAPE

In [34]:
for forum, forum_id in zip(forums, forum_ids):
    
    try:
        
        logger = create_forum_logger(log_path, forum_id)
        
        if first_scrape:
            # HOW LONG WE'LL INITIALLY SCRAPE FOR BEFORE TAKING A BREAK
            start_time, scrape_time = get_next_scrape_times(min_time, max_time)
            bed_time, sleep_time = get_bed_time(22, 3)
            first_scrape = False
        
        threads = get_thread_list(forum)

        for thread in threads:
        
            thread['thread_url'] = get_thread_url(thread['thread_id'], 0, 40)
            thread['num_pages'] = get_num_thread_pages(thread['thread_url'], headers)
            
            log_thread_report('scraping', thread)

            for page in range(thread['num_pages']):
                
                thread_page_url = get_thread_url(thread['thread_id'], page, 40)
                thread_page = s.get(thread_page_url, headers=headers)
                
                if not thread_page.raise_for_status():
                    
                    log_thread_report('retrieved', thread, page, response=thread_page) # TODO: MOVE LOG INTO SAVE HTML
                    save_html_file(thread_page, out_path, thread, page)
                                        
                    if take_breaks and time_to_take_a_break(start_time, scrape_time):

                        wait_time = random.randint(20, 30) * 60
                        send_progress_report('downtime', thread['forum_id'], wait_time)
                        start_time, scrape_time = get_next_scrape_times(min_time, max_time, wait_time, seconds=True)
                        
                    elif go_to_bed and time_for_bed(bed_time):
                        
                        wait_time = sleep_time # TODO: FIX REDUNDANCY
                        send_progress_report('bedtime', thread['forum_id'], wait_time)
                        bed_time, sleep_time = get_bed_time(22, 3)
                    
                    else:
                        wait_time = random.uniform(0.5, 2.0)

                    time.sleep(wait_time)

            log_thread_report('completed', thread, page)

        send_progress_report('forum_complete', forum_id)

    except Exception as err:
        
        report_sent = send_critical_report(err, traceback)

        raise
        
send_progress_report('finished', forum_id, total_forums=len(forum_ids))

## FUNCTIONS

### SCRAPING

In [32]:
# https://www.peterbe.com/plog/best-practice-with-retries-with-requests
# https://stackoverflow.com/questions/35704392/how-to-make-python-post-requests-to-retry
def create_session(retries=3, backoff=0.3, forcelist=(500,502,503,504,521), session=None):
    
    session = session or requests.Session()
    
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff,
        status_forcelist=forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    return session

# s = create_session(10)

# t0 = time.time()
# try:
#     response = s.get('http://localhost:9999')
# except Exception as x:
#     print('It failed :(', x.__class__.__name__)
# else:
#     print('It eventually worked', response.status_code)
# finally:
#     t1 = time.time()
#     print('Took', t1 - t0, 'seconds')

#### GET THREAD LIST

In [98]:
def get_thread_list(csv_filename, logger=True):
    """
    Given a path to a csv file which lists all threads in a forum
    Returns a list of dicts
    Logs "LOADED X THREADS FROM: path/to/csv_filename.csv"
    
    Parameters
    ----------
    csv_filename : str
        The path and filename to the csv file
        
    Returns
    -------
    thread_list : list
        A list of dicts where each dict is one line of the csv file
        
    Examples
    --------
    >>> csv_filename = '../some/path/threads.csv'
    >>> thread_list = get_thread_list(csv_filename)
    >>> thread_list[0]
    
    {'forum_id': '65',
     'num_thread_replies': '127',
     'num_thread_views': '38,017',
     'thread_id': '77500',
     'thread_title': 'Welcome to the Juvenile Forum-Introduce yourself'}
    
    """
    
    thread_list = []
    
    with open(csv_filename) as csv_file:
        
        reader = csv.DictReader(csv_file)
        
        for thread in reader:
                        
            thread_list.append(thread)
    
    if logger:
        logger.info('LOADED ' + str(len(thread_list)) + ' THREADS FROM: ' + csv_filename)

    return thread_list

# csv_filename = '../examples/threads/thread_lists/thread_list_65.csv'
# thread_list = get_thread_list(csv_filename)

#### GET THREAD URL

In [34]:
def get_thread_url(thread_id, start_page, posts_per):
    
    """
    Given a thread id, starting page, and number of pages to show at a time, returns a URL
    
    Parameters
    ----------
    thread_id : int
        The desired thread to retrieve
    start_page : int
        The page in the thread to retrieve
    pages_per : int
        How many pages posts to display at a time
        
    Returns
    -------
    str
        A url to retrieve the page of posts of the thread requested
        
    Examples
    --------
    >>> get_thread_url(671868, 0, 40)
    'http://prisontalk.com/forums/showthread.php?t=671868&order=DESC&pp=40&page=1'
    
    """

    thread = str(thread_id)
    order = 'DESC'
    pp = str(posts_per) # THE MAX IS 40 FOR THREADS
    page = str(start_page + 1) # SINCE INEVITABLE ARRAY STARTS AT THE 0th ELEMENT, ADD 1

    thread_url = 'http://prisontalk.com/forums/showthread.php?'+ \
                                                't='+thread+ \
                                                '&order='+order+ \
                                                '&pp='+pp+ \
                                                '&page='+page
    return thread_url

# 'http://prisontalk.com/forums/showthread.php?t=671868&order=DESC&pp=40&page=1'
# get_thread_url(671868, 0, 40)

#### GET NUM THREAD PAGES

In [35]:
def get_num_thread_pages(thread_url, headers):
    """
    Given request headers and a URL to a thread in a forum, returns the number of pages of replies in the thread
    NOTE: Request session must already be started elsewhere
    
    Parameters
    ----------
    thread_url : str
        The URL location of page one of threads to retreive
    headers: dict
        The headers to be used for requesting the page over the web
        
    Returns
    -------
    num_pages : int
        The number of pages of replies in the thread
        
    Examples
    --------
    >>> thread_url = 'http://prisontalk.com/forums/showthread.php?t=671868&order=DESC&pp=40&page=1'
    >>> headers = {'Host': 'domain.com', ... 'Connection': 'keep-alive'}
    >>> 'NO. OF PAGES:', get_num_thread_pages(thread_url, headers)
    NO. OF PAGES: 4
    """
    
    try:
        initial_thread_page = s.get(thread_url, headers=headers)
        
        thread_page = str(BeautifulSoup(initial_thread_page.content, 'html.parser'))
        no_literals = thread_page.replace('\n', '').replace('\t', '').replace('\r', '')
        thread_page = BeautifulSoup(no_literals, 'html.parser')# skip u'html PUBLIC "-//W3C//
    
        # THERE IS A TOP AND BOTTOM NAV SO WE'LL JUST TAKE THE TOP
        nav = thread_page.find_all('div', {'class':'pagenav'})[0]
        page_x_of_y = nav.find_all('td', {'class':'vbmenu_control'})[0]
        num_pages = page_x_of_y.text.split(' ')[-1]
        
    except IndexError as er:
        
        num_pages = 1
    
    return int(num_pages)

# url = get_thread_url(671868, 0, 40)
# get_num_thread_pages(url, headers) # 4

#### SAVE HTML FILE

In [36]:
def save_html_file(thread_page, path, thread, page):
    """
    Given a http response, path, thread dict, and page number
    Saves response.contents as dirpath/threads/fid/fid_X_tid_Y_pg_Z.html
    Logs MM/DD/YYYY HH:MM:SS PM INFO: SAVED: dirpath/threads/fid/fid_X_tid_Y_pg_Z.html
    
    Parameters
    ----------
    thread_page : response
        The response from a http request
    path : str
        The path to save the page to
    thread: dict
        A dict that contains at least the following keys: forum_id, thread_id
    page : int
        The number of the current page retrieved in the thread

    Returns
    -------
    forum_path + file_name : str
        The path and filename of the file saved
        
    Examples
    --------
    >>> save_html_file(response, 'path/to/dir/', thread, 0)
    """
    
    fid = str(thread['forum_id'])
    tid = str(thread['thread_id'])
    pg = str(page + 1)
    
    # NEW DIRECTORY PATH, EX: 'data/forums/28/'
    forum_path = path + fid + '/'
    # NEW FILENAME, EX: 'fid_39_pg_14.html 
    file_name = 'fid_' + fid + '_tid_' + tid + '_pg_' + pg + '.html'
    
    # CREATE FORUM_PATH DIRECTORY IF IT DOESN'T EXIST, ELSE, MOVE ON
    Path(forum_path).mkdir(parents=False, exist_ok=True)

    #SAVE FILE
    with open(forum_path + file_name, 'w') as html_file:

        html_file.write(thread_page.content)
        
    logger.info('SAVED ' + forum_path + file_name)

# save_html_file(thread_page, out_path, threads[0], 0)
# # '../examples/threads/65/fid_65_tid_77500_pg_1.hmtl'

#### TIME TO TAKE A BREAK

In [37]:
def time_to_take_a_break(start_time, minutes_to_scrape):
    """
    Given the time scraping had begun, and the number of minutes for which scraping should go on for
    Obtains the difference between the two
    Returns True if enough time has elapsed (difference is zero)
    Returns False if there is still time to go, (difference is greater than zero)
    
    Parameters
    ----------
    start_time : datetime.datetime
        Time the scraping began
    minutes_to_scrape : int
        The time in minutes the scraping should last
        
    Returns
    -------
    elapsed_time.total_seconds() >= seconds_to_scrape : bool
        True if elapsed time exceeds the time in seconds that scraping was to take place for
        
    Examples
    --------
    >>> five_seconds = 0.08343145
    >>> eight_seconds = 0.1341245
    >>> start = datetime.now()
    >>> time.sleep(5)
    >>> time_to_take_a_break(start, five_seconds), time_to_take_a_break(start, eight_seconds)
    (True, False)
    """
    
    current_time = datetime.now()
    # NOT REALLY NECCESSARY IN MINUTE CONDITIONS BUT TESTS ARE IN SECONDS AND I'M IMPATIENT
    seconds_to_scrape = int(minutes_to_scrape * 60.0) 
    
    elapsed_time = current_time - start_time
    break_time = elapsed_time.total_seconds() >= seconds_to_scrape
    # print 'BREAKTIME:', break_time, '--> SECONDS ELAPSED:', elapsed_time.total_seconds(), 'SECONDS TO SCRAPE:', seconds_to_scrape
    
    return break_time

# five_seconds = 0.08343145 # ~5.005 seconds, returns True
# eight_seconds = 0.1341245 # ~8.047 seconds, returns False
# start = datetime.now()
# time.sleep(5)
# time_to_take_a_break(start, five_seconds), time_to_take_a_break(start, eight_seconds) # (True, False)

#### GET NEXT SCRAPE TIMES

In [38]:
def get_next_scrape_times(min_time, max_time, additional_time=0, seconds=False):
    """
    Given a minimum and maximum time in minutes, returns the current time, 
    and a random length of time in minutes to scrape for.
    Used for calculating the difference between the two function time_to_take_a_break()
    Additionally, you can factor in some additional_time to compensate (e.g. time spent sleeping)
    
    Parameters
    ----------
    min_time : int
        Minimum time to scrape for in minutes
    max_time : int
        Maximum time to scrape for in minutes
    additional_time : int
        Optional - An ammount of time to add in minutes if seconds is False (see below), default is zero 
    seconds : boolean
        Optional - What time scale the additional_time is in, default is False (e.g. minutes)
        
    Returns
    -------
    datetime.datetime
        The current time plus any additional time
    int
        The time to scrape for in minutes
        
    Examples
    --------
    >>> start_time, scrape_time = get_next_scrape_times(8, 18, 3)
    >>> print 'WAIT:', wait_time, '\n', 'NOW:', datetime.now(), '\n', 'BEGIN NEXT SCRAPE:', start_time, '\n', 'NEXT SCRAPE LENGTH:', scrape_time, 'minutes'
    WAIT: 3 
    NOW: 2018-06-20 13:36:14.822685 
    BEGIN NEXT SCRAPE: 2018-06-20 13:39:14.821660 
    NEXT SCRAPE LENGTH: 17 minutes
    """
    
    wait_time = additional_time * 60 if seconds == False else additional_time
    
    next_start_time = datetime.now().replace(microsecond=0) + timedelta(seconds=wait_time) # TO SUBTRACT LATER
    next_scrape_length = random.randint(min_time, max_time) # HOW LONG WE'LL SCRAPE FOR BEFORE TAKING A BREAK
    
    logger.info('NEXT SCRAPE: ' + str(next_start_time) + ' DURATION: ' + str(next_scrape_length) + ' minutes')

    return next_start_time, next_scrape_length

# wait_time = random.randint(6, 10)
# start_time, scrape_time = get_next_scrape_times(8, 18, wait_time)
# print 'WAIT:', int(wait_time), 'minutes' '\n', 'NOW:', datetime.now(), '\n', 'BEGIN NEXT SCRAPE:', start_time, '\n', 'NEXT SCRAPE LENGTH:', scrape_time, 'minutes'
# print ''
# wait_time = random.randint(6, 10) * 60
# start_time, scrape_time = get_next_scrape_times(8, 18, wait_time, seconds=True)
# print 'WAIT:', int(wait_time / 60.0), 'minutes' '\n', 'NOW:', datetime.now(), '\n', 'BEGIN NEXT SCRAPE:', start_time, '\n', 'NEXT SCRAPE LENGTH:', scrape_time, 'minutes'

#### TIME FOR BED

In [39]:
def time_for_bed(bed_time):
    """
    Given a date and time when the program should go to sleep for the evening
    Returns True if the current time is past bedtime, False if not.
    
    Parameters
    ----------
    bed_time : datetime.datetime
        The time when sleep should occur
        
    Returns
    -------
    datetime.now() > bed_time : bool
        True if current time is past bedtime, False if not.
        
    Examples
    --------
    >>> # SUPPOSE CURRENT TIME IS 2018-7-2 14:06:00
    >>> bed_time = datetime.datetime(2018, 7, 2, 22, 2, 15)
    >>> time_for_bed(bed_time)
    False
    """
    
    return datetime.now() > bed_time

# before_bedtime = datetime.now() + timedelta(seconds=5)
# past_bedtime = datetime.now() - timedelta(seconds=5)
# print time_for_bed(before_bedtime) # FALSE
# time.sleep(5)
# print time_for_bed(past_bedtime) # TRUE

#### GET BED TIMES

In [40]:
def get_bed_time(min_bedtime, max_bedtime, min_hours=7, max_hours=9, debug=False):
    """
    Given a minimum hour (0 - 23) and a maximum hour (0 - 23), chooses a datetime with a random hour
    within that range.  If range of hours goes from before midnight to after midnight (e.g. 10, 11, 0, 1, 2), 
    the datetime returned is shifted by 1 day if the hour chosen is after midnight (i.e. > 0).  
    Also returns a random time in seconds for which to 'sleep'.
    
    Parameters
    ----------
    min_bedtime : int
        The earliest hour in miltary time that sleep should begin
    max_bedtime : int
        The latest hour in military time that sleep should begin
    min_hours : int
        Optional, default 7.  Minimum length of time to sleep.
    max_hours : int
        Optional, default 9.  Maximum length of time to sleep.
    debug : bool
        Optional, default False.  Prints statements about variables.
        
    Returns
    -------
    bedtime : datetime
        The time when sleep should begin
    sleeptime : int
        The length of time in seconds that the sleep should last 
        
    Examples
    --------
    >>> datetime.now().replace(microsecond=0)
    datetime.datetime(2018, 7, 1, 9, 2, 28)
    >>> get_bed_time(22, 2) # HOUR CHOSEN WAS 2
    (datetime.datetime(2018, 7, 2, 1, 55, 6), 29342)
    >>> str(timedelta(seconds=29342))
    '8:09:02'
    """

    if min_bedtime > max_bedtime:
        hours = range(min_bedtime, 24) + range(max_bedtime)
        range_over_midnight = True
    else:
        hours = range(min_bedtime, max_bedtime)
        range_over_midnight = False
    
    hour = random.choice(hours)
    minutes = random.randrange(0, 60)
    shift = timedelta(days=1) if (range_over_midnight) & (hour <= max_bedtime) else timedelta(days=0)

    bedtime = datetime.now().replace(hour=hour, minute=minutes, second=0, microsecond=0) + shift
    sleeptime = random.randrange(min_hours*60*60, max_hours*60*60)
    
    if debug:
        print 'HOUR:', hour, '  RANGE OVER MIDNIGHT:', range_over_midnight, 
        print '  HOUR <= max_bedtime:', (hour <= max_bedtime),
        print '  SHIFT:', shift
        print ''
    
    logger.info('SHIFT: ' + str(bool(shift)) + ' NEXT BEDTIME: ' + str(bedtime) + ' SLEEPTIME: ' + str(timedelta(seconds=sleeptime)))

    return bedtime, sleeptime

# bed_time, sleep_time = get_bed_time(18, 23, debug=True)
# NOW: 2018-07-01 09:07:03 	BEDTIME: 2018-07-01 22:56:06 	SLEEPTIME: 8:44:16
# bed_time, sleep_time = get_bed_time(22, 3, debug=True)
# NOW: 2018-07-01 09:12:02 	BEDTIME: 2018-07-02 02:38:06 	SLEEPTIME: 7:39:28 # IF CHOICE > 0

# print 'NOW:', datetime.now().replace(microsecond=0), '\tBEDTIME:', bed_time, '\tSLEEPTIME:', str(timedelta(seconds=sleep_time)) 

### REPORTING

#### CREATE FORUM LOGGER

In [41]:
def create_forum_logger(file_path, forum_id):
    """
    Given a file path and thread dict 
    Returns a logger which logs to that path with the file forum_X.log in the following format
    18/06/2018 05:27:11 PM INFO: SOME TEST MSG
    
    Parameters
    ----------
    file_path : str
        The path to where the log should be written to
    forum_id : int
        The interger id of the your are going to scrape

    Returns
    -------
    logger : logger
        The logger which saves logs to the file forum_X.log
        
    Examples
    --------
    >>> logger = create_forum_logger('some/path/to/log/', thread_dict)
    >>> logger.info('SOME TEST MSG')
    """

    log_file = 'forum_'+ str(forum_id) + '.log'
    msgfmt = '%(asctime)s %(levelname)s: %(message)s'
    datefmt= '%d/%m/%Y %I:%M:%S %p'
    
    reload(logging)
    logging.basicConfig(filename=file_path + log_file, level=logging.INFO, format=msgfmt, datefmt=datefmt)
    logging.Formatter.converter = time.gmtime # CHANGE TO UTC TIME
    logger = logging.getLogger()
    
    return logger 

# log_path = '../examples/threads/logs/'
# fid = 0
# logger = create_forum_logger(log_path, fid)
# logger.info('SOME TEST MSG') # logs 18/06/2018 05:27:11 PM INFO: SOME TEST MSG to forum_0.log

#### LOG THREAD REPORT

In [42]:
def log_thread_report(status, thread, current_page=0, response=None):
    """
    Given a thread dict and a status type
    Logs a status report to the thread log
    18/06/2018 05:27:11 PM INFO: RETRIEVING FORUM: 65 THREAD: 77500 REPLIES: 122 PAGE: 0 OF 4
    
    Parameters
    ----------
    status: str
        A string which indicates which message to log: 'retreiving', 'completed'
    thread : dict
        A dict that includes the keys: forum_id, thread_id, num_pages, num_thread_replies
    current_page : int
        Current page being retreived
    *args : Added parameters for various messages
    
    Returns
    -------
        None
    
    Examples
    --------
    >>> log_thread_report('scraping', thread, page)
    """
    
    fid = str(thread['forum_id'])
    tid = str(thread['thread_id'])
    rpls = str(thread['num_thread_replies'])
    pg = str(current_page + 1)
    pgs = str(thread['num_pages'])
    
    if response:
        query = response.url.split('/')[-1]
        status_code = str(response.status_code)

    if status == 'scraping':
        report = status.upper()+' FORUM: '+fid+' THREAD: '+tid+ ' REPLIES: '+rpls+' PAGES: '+pgs
    elif status == 'retrieved':
        report = status.upper()+' THREAD: '+tid+' PAGE: '+pg+' OF '+pgs+' FROM: '+query+' STATUS: '+status_code
    elif status == 'completed':
        report = status.upper()+' SCRAPING '+pg+' OF '+pgs+' PAGES FROM FORUM: '+fid+' THREAD: '+tid+ ' REPLIES: '+rpls
        
    logger.info(report)
    # print report

# logger = create_forum_logger(log_path, threads[0])
# # 18/06/2018 06:25:18 PM INFO: RETREIVING FORUM: 65 THREAD: 77500 REPLIES: 127 PAGE: 3 OF 4
# url_str = 'http://prisontalk.com/forums/showthread.php?t=671868&order=DESC&pp=40&page=1'
# test_page = s.get(url_str, headers=headers)
# log_thread_report('scraping', threads[0])
# log_thread_report('retrieved', threads[0], 0, response=test_page)
# log_thread_report('completed', threads[0], 3)

**NOTE:** *YOU MUST FILL IN THE FOLLOWING VARIABLES IN THE TWO REPORT FUNCTIONS **OR** ADD THEM TO A [ConfigParser](https://docs.python.org/2/library/configparser.html) TXT FILE TO BE IMPORTED BY LOAD_CONFIG (SEE HELPERS BELOW)*

1. email_address=where_to_send_report@domain.com
2. gmail_address=gmail_address@gmail.com
3. gmail_password=gmail_password
4. phone_email_address=XXXYYYZZZZ@txt.phoneco.net

#### SEND PROGRESS REPORT

In [43]:
def send_progress_report(progress, forum_id, wait_time=0, total_forums=0, text=False):
    
    creds = load_config('report_credentials.txt')
    
    email_address = creds['email_address']
    gmail_address = creds['gmail_address']
    gmail_password = creds['gmail_password']
    phone_email_address = creds['phone_email_address']
        
    try:

        if progress == 'forum_complete':

            headline = 'FINISHED SCRAPING LIST OF THREADS FROM FORUM ' + str(forum_id) #+ '\r\n'
            other_info = '' # MAYBE ADD HOW MANY FORUMS ARE LEFT?
            closing = '\r\n' + 'TALK SOON!'

        elif progress =='downtime':

            headline = 'TAKING A BREAK FOR ' + str(int(wait_time/60.0)) + ' MINUTES TO KEEP UNDER THE RADAR' #+ '\r\n'
            other_info = '\r\n'+ 'STILL DOWNLOADING THREADS FROM FORUM ' + str(forum_id) #+ '\r\n'
            closing = '\r\n' + 'BE SEEING YOU...!'
            
        elif progress =='bedtime':

            headline = 'PACKING IT IN FOR THE EVENING. CURRENT TIME IS: ' + str(datetime.now().replace(microsecond=0))
            other_info = '\r\n' + 'GOING TO SLEEP FOR: ' + str(timedelta(seconds=wait_time))
            closing = '\r\n' + 'WILL BE BACK UP AROUND: ' + str(datetime.now().replace(microsecond=0) + timedelta(seconds=wait_time)) + '\r\nNIGHT!'
            
        elif progress =='continuing':

            headline = 'OKAY. GOING BACK AT IT.'
            other_info = '\r\n' + 'CURRENT TIME IS: ' + str(datetime.now().replace(microsecond=0))
            closing = '\r\n' + 'I\'LL KEEP YOU POSTED.'

        elif progress == 'finished':

            headline = 'IT IS FINISHED: ALL ' + str(total_forums) + ' LISTS OF THREADS DOWNLOADED' #+ '\r\n'
            other_info = '\r\n' + 'LAST FORUM DOWNLOADED: ' + str(forum_id) #+ '\r\n'
            closing = '\r\n' +'I\'M PRETTY CACHED, AND GRATEFUL WE DIDN\'T GET BANNED. PEACE.'

        else:

            headline = 'AN UNKNOWN PROGRESS STATUS WAS PASSED: ' + progress #+ '\r\n'
            other_info = '\r\n' + 'CONTINUING TO DOWNLOAD THREADS FROM FORUM: ' + str(forum_id) #+ '\r\n'
            closing = '\r\n' + 'SORRY.'

        logger.warning(headline)

        email_msg = MIMEMultipart()

        email_msg['From']=' YOUR SCRAPER'
        email_msg['To'] = email_address
        email_msg['Subject'] = 'PRISONTALK: PROGRESS REPORT'
        email_msg.attach(MIMEText(headline + other_info + closing, 'plain'))

        smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)
        smtp_gmail.starttls()
        smtp_gmail.login(gmail_address, gmail_password)

        smtp_gmail.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())

        if text:

            text_msg = MIMEMultipart()

            text_msg['From']= 'YOUR SCRAPER'
            text_msg['To'] = phone_email_address
            text_msg['Subject'] = 'GOOD NEWS EVERYONE!'
            text_msg.attach(MIMEText(progress.upper()))

            smtp_gmail.sendmail(text_msg['From'], text_msg['To'], text_msg.as_string())

        logger.warning('PROGRESS REPORT SENT VIA ' + ('TEXT & EMAIL' if text == True else 'EMAIL') + '!')

        smtp_gmail.close()
        
    except Exception as err:
        
        err_type = err.__class__.__name__
        
        print err_type + ' ' + str(err)

        logger.error('COULDN\'T SEND PROGRESS REPORT: ' + err_type + ' ' + str(err)) 
        logger.error('TRIED TO REACH YOU. GOT VOICEMAIL. I THINK I\'LL JUST QUIT WHILE I\'M AHEAD')
        
        return False 


# forum_id = 65
# seconds = 600
# sleep_time = 26517
# list_of_forums = [65, 105, 122, 320]
# logger = create_forum_logger(log_path, forum_id)

# send_progress_report('downtime', forum_id, seconds)
# send_progress_report('bedtime', forum_id, sleep_time)
# send_progress_report('forum_complete', forum_id, text=True)
# send_progress_report('finished', forum_id, total_forums=len(list_of_forums), text=True)
# send_progress_report('continuing', forum_id, text=True)

#### SEND CRITICAL FAILURE REPORT

In [44]:
def send_critical_report(err, tb):
    
    creds = load_config('report_credentials.txt')
    
    email_address = creds['email_address']
    gmail_address = creds['gmail_address']
    gmail_password = creds['gmail_password']
    phone_email_address = creds['phone_email_address']
    
    try:
        
        logger.critical('CRITICAL ERROR: ' + err.__class__.__name__ + ' ' + str(err))

        text_msg = MIMEMultipart()

        text_msg['From']=' YOUR SCRAPER'
        text_msg['To'] = phone_email_address
        text_msg['Subject'] = ' DON\'T PANIC BUT...'
        text_body = 'CHECK YOUR EMAIL.'

        text_msg.attach(MIMEText(text_body))

        
        email_msg = MIMEMultipart()

        email_msg['From']=' YOUR SCRAPER'
        email_msg['To'] = email_address
        email_msg['Subject'] = 'PRISONTALK: CRITICAL ERROR REPORT'
        
        tb_str = tb.format_exc() # REMOVE IN THE FUTURE AND ADD DIRECTLY TO BODY OR KEEP FOR CLARITY?
        email_body = 'ERROR:\r\n' + \
                    '\r\n' + \
                    tb_str + \
                    '\r\n' + \
                    'TRIED TO REACH YOU.\r\n' + \
                    'GOT VOICEMAIL.\r\n' + \
                    'GET READY FOR THE THUNDERDOME.\r\n' + \
                    'SHUT IT DOWN FOLKS. SHUT IT DOWN.'

        email_msg.attach(MIMEText(email_body, 'plain'))


        smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)
        smtp_gmail.starttls()
        smtp_gmail.login(gmail_address, gmail_password)

        smtp_gmail.sendmail(text_msg['From'], text_msg['To'], text_msg.as_string())
        smtp_gmail.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())
        
        smtp_gmail.close()
        logger.critical('CRITICAL ERROR: REPORT SENT VIA TEXT & EMAIL')
        
        return True
        
    except Exception as err:
                
        logger.critical('DOUBLE CRITICAL ERROR: ' + err.__class__.__name__ + ' ' + str(err)) 
        logger.critical('TRIED TO REACH YOU. GOT VOICEMAIL. SHUT IT DOWN FOLKS! SHUT IT DOWN.')
        
        return False 

# # TEST IN A LOOP TO MAKE SURE IT WILL EXIT AND YOU WON'T BE SPAMMED BY YOUR OWN PROGRAM
# for i in range(2):
    
#     try:
#         1/0
#     except Exception as err:

#         report_sent = send_critical_report(err, traceback)
#         print 'REPORT SENT:', report_sent

#         raise        

# # SHOULD PRINT THE FOLLOWING AND SEND *ONE* TEXT AND EMAIL OF MESSAGE ABOVE
# # REPORT SENT: True
# # ---------------------------------------------------------------------------
# # ZeroDivisionError                         Traceback (most recent call last)
# # ...

### HELPERS

#### LOAD CONFIG FILE

In [45]:
def load_config(filename):
    
    if not os.path.isfile(filename): # CHECK IF PATH/FILE EXISTS
        return None

    config = ConfigParser.ConfigParser() # IF PATH/FILE DOES EXIST PARSE THE CONFIG
    config.read(filename)
    sections = config.sections() # DIVIDE UP INTO SECTIONS OF EACH CONFIG

    for section in sections: # LOOP THROUGH EACH CONFIG SECTION
        
        options = config.options(section) # GET THE OPTIONS OF EACH SECTION

        profile = {}

        for option in options: # FOR EACH OPTION IN A SECTION

            profile[option] = config.get(section, option) # ADD IT TO A PROFILE DICT

    return profile


# IN A .TXT FILE

# [1]
# email_address=where_to_send_report@domain.com
# gmail_address=gmail_address@gmail.com
# gmail_password=gmail_password
# phone_email_address=XXXYYYZZZZ@txt.phoneco.net

# report_creds = load_config('report_credentials.txt')
# report_creds

#### FILTER COMPLETED THREADS

In [186]:
def filter_completed_threads(thread_filename, completed_filename, out_path=None):
    
    with open(thread_filename) as thread_file, open(completed_filename) as completed_file:

        thread_reader = csv.DictReader(thread_file)
        completed_reader = csv.reader(completed_file)

        all_threads = [thread for thread in thread_reader]    
        completed_threads = [completed_thread for completed_threads in completed_reader for completed_thread in completed_threads]

        all_thread_ids = set([thread['thread_id'] for thread in all_threads])
        completed_thread_ids = set([thread_file.split('_')[3] for thread_file in thread_files])

        remaining_thread_ids = list(all_thread_ids.symmetric_difference(completed_thread_ids))
        remaining_threads = [thread for thread in all_threads if thread['thread_id'] in remaining_thread_ids]
        
        if out_path:
            
            with open(out_path, 'w') as outfile:
                
                writer = csv.DictWriter(outfile, fieldnames=remaining_threads[0].keys())
                writer.writeheader()
                
                for remaining_thread in remaining_threads:
                    writer.writerow(remaining_thread)
            
        print len(list(completed_thread_ids)), '+', len(remaining_thread_ids), '=', len(all_threads)
        
        return remaining_threads
    
# # # 2663 + 2765 = 5428, wc -l remaining.126 should equal len(remaining_thread_ids) + 1
# thread_filename = '../examples/threads/thread_lists/thread_list_126.csv'
# completed_filename = '../examples/threads/completed/already_completed_126.txt'
# out_path = '../examples/threads/completed/remaining_126.csv'
# imcomplete = filter_completed_threads(thread_filename, completed_filename)
# write_imcomplete = filter_completed_threads(thread_filename, completed_filename, out_path)

2663 + 2765 = 5428


#### GET RETRIEVAL TIME

In [46]:
def get_retrieval_time(replies, replies_per_page):
    """
    Given a number of threads and the threads per page
    Returns an the time in minutes of how long it will take to download all of the pages
    
    Parameters
    ----------
    replies : str
        The number of threads
    replies_per_page : int
        The number of threads on each page
        
    Returns
    -------
    total_minutes : int
        The total number of minutes it will take to download the pages of replies
        
    Examples
    --------
    >>> get_retrieval_time(4440, 40)
    12
    """
    
    average_page_wait = 7
    
    pages = int(replies)/replies_per_page
    
    total_seconds = pages * 7
    
    total_minutes = total_seconds/60
    
    return total_minutes

# get_retrieval_time(4440, 40) # 12

#### PRINT HEADERS

In [47]:
def print_headers(response):
    
    print 'RESPONSE HEADERS'
    for key, val in response.headers.iteritems():
        print key + ':', val
    print ''
    print 'REQUEST HEADERS'
    for key, val in response.request.headers.iteritems():
        print key + ':', val
        
# s = requests.session()
# test_response = s.get('http://prisontalk.com', headers=headers)
# print_headers(test_response)

# # RESPONSE HEADERS
# # Date: Mon, 26 Feb 2018 20:57:24 GMT
# # Server: Apache/2.2.3 (CentOS)
# # ...
# # Transfer-Encoding: chunked
# # Content-Type: text/html; charset=ISO-8859-1
    
# # REQUEST HEADERS
# # Connection: keep-alive
# # Accept-Encoding: gzip, deflate
# # ...
# # DNT: 1
# # Host: www.prisontalk.com

#### EXCEPTIONS TESTING

In [349]:
# for i in [0,1,2]:
    
#     try:
#         if i == 1:
#             1/0
#         else: 
#             print 'i is equal to', i
        
#     except Exception as err:

# #         print err.__class__.__name__, err
        
# #         print traceback.format_exc()

#         raise

#### BEDTIME TESTING

In [115]:
spans_midnight, sleep_time = get_bed_time(22, 3)
print spans_midnight, str(timedelta(seconds=sleep_time))

'7:22:28'

In [ ]:
# WHILE TRUE
# CHECK IF IT'S BEDTIME
# IF IT'S BEDTIME THEN GO TO SLEEP FOR SLEEPTIME
# WHEN YOU WAKE UP, TURN OFF THE WHILE LOOP
# ELSE, SLEEP FOR A SMALLER TIME

log_path = '../examples/threads/logs/'
forum_id = 143

logger = create_forum_logger(log_path, forum_id)

# spans_midnight, sleep_time = get_bed_time(22, 3)
# BEDTIME: 2018-07-03 00:09:00 
# SLEEPTIME: '7:22:28' (26548 seconds)

not_rested = True

while not_rested:
    
    if time_for_bed(spans_midnight):

        logger.info('IT\'S PAST MY BEDTIME: '+ str(spans_midnight) + ' GOING TO SLEEP FOR: ' + str(timedelta(seconds=sleep_time)))
        logger.info('WILL WAKE AT: ' + str(datetime.now().replace(microsecond=0) + timedelta(seconds=sleep_time)))

        send_progress_report('bedtime', forum_id, sleep_time)

        time.sleep(sleep_time)

        send_progress_report('continuing', forum_id)

        spans_midnight, sleep_time = get_bed_time(22, 3)
        logger.info('I\'M UP! I\'M UP! GEEEEEEEZ. I\'LL GO TO BED NEXT AT ' + str(spans_midnight) + ' FOR ' + str(timedelta(seconds=sleep_time)))

        not_rested = False

    else:

        logger.info('CURRENT TIME: ' + str(datetime.now().replace(microsecond=0)) + ' STILL ' + str(spans_midnight - datetime.now().replace(microsecond=0)) + ' UNTIL BED')
        time.sleep(900)

In [ ]:
03/07/2018 06:51:33 AM INFO: CURRENT TIME: 2018-07-02 23:51:33 STILL 0:17:27 UNTIL BED
03/07/2018 07:06:33 AM INFO: CURRENT TIME: 2018-07-03 00:06:33 STILL 0:02:27 UNTIL BED
03/07/2018 07:21:33 AM INFO: IT'S PAST MY BEDTIME: 2018-07-03 00:09:00 GOING TO SLEEP FOR: 7:22:28
03/07/2018 07:21:33 AM INFO: WILL WAKE AT: 2018-07-03 07:44:01
03/07/2018 07:21:33 AM INFO: PACKING IT IN FOR THE EVENING.
CURRENT TIME IS: 2018-07-03 00:21:33
03/07/2018 07:21:35 AM INFO: PROGRESS REPORT SENT VIA EMAIL!
03/07/2018 02:44:03 PM INFO: OKAY. GOING BACK AT IT.
03/07/2018 02:44:05 PM INFO: PROGRESS REPORT SENT VIA EMAIL!
03/07/2018 02:44:05 PM INFO: I'M UP! I'M UP! GEEEEEEEZ. I'LL GO TO BED NEXT AT 2018-07-03 22:59:00 FOR 7:22:30